In [1]:
import os, shutil
import sys
from tensorflow.keras.utils import disable_interactive_logging
import time
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import Callback
import os
import random
import numpy as np
import tensorflow as tf
from pymannkendall import seasonal_test
from sklearn.preprocessing import MinMaxScaler, StandardScaler


from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras.layers import Conv1D, LSTM, Lambda, Dropout,Bidirectional
from keras_tuner import RandomSearch
from keras.optimizers import Adam
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from keras_tuner import HyperParameters
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import math
from keras.utils import plot_model

In [2]:
def CNN_train(training_ratio,look_back,batch_size,file):
    disable_interactive_logging()
    class Tee:
        def __init__(self, *files):
            self.files = files
        def write(self, obj):
            for f in self.files:
                f.write(obj)
                f.flush()
        def flush(self):
            for f in self.files:
                f.flush()

    sys.stdout = Tee(sys.stdout, sys.stderr)

    directory=f'lstm_{training_ratio}_{look_back}_{batch_size}'
    if os.path.exists(directory):    
        shutil.rmtree(directory)
        print(f"{directory} is removed successfully")


    start_time = time.time()

    # load a dataset
    data_file = f"../Data/WekaData/{file}.arff"
    # Load arff file
    data, meta = arff.loadarff(data_file)
    data_df = pd.DataFrame(data)



    # Convert to pandas DataFrame
    #data_df = pd.read_csv('bitcoin_hist.csv')
    #data_df["Date"] =  pd.to_datetime(data_df["Date"], format="%m/%d/%Y")

    # data_df.plot(x='Date', y=['Close', 'Volume'], style=['r-', 'g-'], figsize=(10, 6))  # for stock

    # # define the date format
    # date_form = mdates.DateFormatter('%Y')

    # # set the x-axis major locator to every even year
    # ax = plt.gca()
    # ax.xaxis.set_major_locator(mdates.YearLocator(base=1))
    # ax.xaxis.set_major_formatter(date_form)
    # ax.set_yscale("log")

    # plt.show()


    close_index = data_df.columns.get_loc('Close')
    dataset = data_df.iloc[:, close_index:close_index+1].values  # numpy array
    dataset = dataset.astype('float32')



    # define early stopping
    early_stopping5 = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

    class StopAtThreshold(Callback):
        def __init__(self, monitor='loss', threshold=0.01):
            super(StopAtThreshold, self).__init__()
            self.monitor = monitor
            self.threshold = threshold

        def on_epoch_end(self, epoch, logs=None):
            current = logs.get(self.monitor)
            if current is not None and current < self.threshold:
                self.model.stop_training = True



    # create an instance of our custom callback
    stop_at_threshold = StopAtThreshold(monitor='val_loss', threshold=0.015)

    def set_seeds(seed):
        os.environ['PYTHONHASHSEED'] = str(seed)
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        # If you are using CUDA, uncomment the following 2 lines
        # os.environ['TF_DETERMINISTIC_OPS'] = '1'
        # os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    set_seeds(1234)

    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)

    result = seasonal_test(data_df['Close'])
    print(result)
    trend, h, p, z, Tau, s, var_s, slope, intercept = result
    import re
    params=re.split(r'\s*,\s', "trend, h, p, z, Tau, s, var_s, slope, intercept")
    for pr in params:
        print(f'{pr}={eval(pr)}')



    # because it's multiplicative, so apply np.log
    dataset = np.log(dataset)

    # Initialize a scaler for the dataset
    #scaler = MinMaxScaler(feature_range=(0, 1))
    # Z-score normalization is useful when the data has outliers or when the distribution of the data is not known. 
    scaler = StandardScaler() 

    # Fit and transform the data to the scaler
    # Split into train and test sets
    from sklearn.model_selection import train_test_split
    training_ratio = training_ratio
    train_data, test_data = train_test_split(dataset, train_size=training_ratio, shuffle=False)

    # Fit the scaler to the training data and transform the training data
    train = scaler.fit_transform(train_data)

    # Use the same scaler to transform the test data
    test = scaler.transform(test_data)
    # print(train.shape, test.shape)### Using Multiple Layer Perceptron



    # reshape dataset
    look_back = look_back
    trainX, trainY = create_dataset(train, look_back)
    test_data_with_look_back = np.concatenate((train[-look_back:], test))

    # Create testing data, starting with the end of the training data
    testX, testY = create_dataset(test_data_with_look_back, look_back)

    def build_model(hp):
       # Input layer   
        model = Sequential()  
        # model.add(Conv1D(filters=hp.Int('input_units',min_value=32,max_value=256,step=32)
        #                 ,kernel_size=(look_back),activation='relu',input_shape=[look_back, 1]))
        model.add(Conv1D(filters=160,kernel_size=(look_back),activation='relu',input_shape=[look_back, 1]))
        model.add(Flatten())
        model.add(Dense(1))
        # Output layer    
        model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-1, 1e-2, 1e-3])),loss='mean_absolute_error')

        modelname=f'CNN_{file}_{training_ratio}tr{look_back}lb{batch_size}bs_model.png'
        tf.keras.utils.plot_model(model,show_shapes=True,show_layer_names=True, show_layer_activations=False,to_file=modelname)
        return model

    # create a TimeSeriesSplit object
    tscv = TimeSeriesSplit(n_splits=5)

    tuner = RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=5,
        executions_per_trial=3,
        directory=directory,
        project_name='bitcoin')

    # define early stopping
    early_stopping15 = EarlyStopping(monitor='val_loss', patience=15, verbose=0)
    # create an instance of our custom callback
    stop_at_threshold = StopAtThreshold(monitor='val_loss', threshold=0.01)
    # perform hyperparameter tuning with time series cross-validation
    for train_index, val_index in tscv.split(trainX):
        X_train, X_val = trainX[train_index], trainX[val_index]
        y_train, y_val = trainY[train_index], trainY[val_index]
        tuner.search(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=10,
            callbacks=[early_stopping15]
            #callbacks=[stop_at_threshold]
        )

    # tuner.search_space_summary()
    # get the best hyperparameters
    best_hp = tuner.get_best_hyperparameters()[0]

    # get the best trial
    best_trial = tuner.oracle.get_best_trials()[0]



    # get the score of the best trial
    best_score = best_trial.score

    # # print the score of the best trial
    # print(f"Best score: {best_score}")

    # print the values of the best hyperparameters
    # for hp in best_hp.values:
    #     print(f"{hp}: {best_hp.get(hp)}")



    # define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0)
    # create an instance of our custom callback
    stop_at_threshold = StopAtThreshold(monitor='val_loss', threshold=0.01)



    ntrainX, valX, ntrainY, valY = train_test_split(trainX, trainY, test_size=0.1, shuffle=False)

    #start_time = time.time()

    # create a new HyperParameters object
    new_hp = HyperParameters()

    # set the hyperparameters to the desired values
    new_hp.Fixed('input_units', 196)
    new_hp.Fixed('learning_rate', 0.001)

    # build a new model with the specified hyperparameters
    model = build_model(new_hp)

    # build the best model
    # model = build_model(best_hp)

    # fit the model with early stopping
    history = model.fit(
        ntrainX, ntrainY,
        validation_data=(valX, valY),
        epochs=1000,
        batch_size=batch_size, 
        verbose=0,
        callbacks=[early_stopping15]
        #callbacks=[stop_at_threshold]
    )

    # generate predictions for training
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # Inverse transform the predictions to original scale
    trainPredict_orig = np.exp(scaler.inverse_transform(trainPredict))
    trainY_orig = np.exp(scaler.inverse_transform([trainY]))
    train_mse = mean_squared_error(trainY_orig[0], trainPredict_orig[:,0])
    train_mae = mean_absolute_error(trainY_orig[0], trainPredict_orig[:,0])
    

    testPredict_orig = np.exp(scaler.inverse_transform(testPredict))
    testY_orig = np.exp(scaler.inverse_transform([testY]))

    # Now you can calculate your evaluation metrics on the original scale
    test_mse = mean_squared_error(testY_orig[0], testPredict_orig[:,0])
    test_mae = mean_absolute_error(testY_orig[0], testPredict_orig[:,0])
    
    end_time = time.time()
    elapse = end_time-start_time
    elapse=f'{int(elapse//60)}m {int(elapse%60)}s'
    # print(f'Total time: {elapse//60} minutes, {elapse%60:.4f} seconds.')

    return best_score,best_hp.values['input_units'],best_hp.values['learning_rate'],elapse,math.sqrt(train_mse),train_mae,math.sqrt(test_mse),test_mae


In [3]:
# load a dataset
# data_file=['BTCUSD-all','BTCUSD-N2Y','BTCUSD-N4Y',
#            'ETHUSD-all','ETHUSD-N2Y','ETHUSD-N4Y',           
#           'USDTUSD-all','USDTUSD-N2Y','USDTUSD-N4Y',  
#            'BNBUSD-all','BNBUSD-N2Y','BNBUSD-N4Y']
# data_file=['BTCUSD-1m1h','ETHUSD-1m1h','USDTUSD-1m1h','BNBUSD-1m1h']
data_file=['BTCUSD-all']
training_ratio=[0.7]
look_back=[7]
batch_size=[32]


data_file_list= list()
training_ratio_list= list()
look_back_list= list()
batch_size_list= list()

best_score_list= list()
input_units_list= list()
learning_rate_list= list()
elapse_list= list()

train_rmse_list= list()
train_mae_list= list()
test_rmse_list= list()
test_mae_list= list()



for tr in training_ratio:
    for lb in look_back:
        for bs in batch_size:
            for dataf in data_file:
                res=CNN_train(tr,lb,bs,dataf)

                data_file_list.append(dataf)
                training_ratio_list.append(tr)
                look_back_list.append(lb)
                batch_size_list.append(bs)

                best_score_list.append(res[0])
                input_units_list.append(res[1])
                learning_rate_list.append(res[2])
                elapse_list.append(res[3])
                
                train_rmse_list.append(res[4])
                train_mae_list.append(res[5])
                test_rmse_list.append(res[6])
                test_mae_list.append(res[7])



Trial 3 Complete [00h 00m 03s]

Trial 3 Complete [00h 00m 03s]

val_loss: 0.12499222904443741

val_loss: 0.12499222904443741

Best val_loss So Far: 0.024088069175680477

...

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


KeyError: 'input_units'

In [4]:

Summary={'Data':data_file_list,'training ratio':training_ratio_list,'look back':look_back_list,'batch_size_list':batch_size_list,
         'best_score':best_score_list,'input units':input_units_list,'learning_rate':learning_rate_list,'time train':elapse_list,
            'train_mae':train_mae_list,          
            'train_rmse':train_rmse_list,          
            'test_mae':test_mae_list,           
            'test_rmse':test_rmse_list
            
         }

df_Summary = pd.DataFrame(Summary)
# df_Summary.to_excel("Summary-BiLSTM(1m1g).xlsx",index=False)  
df_Summary

,Data,training ratio,look back,batch_size_list,best_score,input units,learning_rate,time train,train_mae,train_rmse,test_mae,test_rmse
0,BTCUSD-all,0.7,1,4,0.018247,256,0.01,0m 54s,201.946225,480.794411,899.292595,1345.083957
1,BTCUSD-all,0.7,1,8,0.018247,256,0.01,0m 39s,226.476462,738.835559,2793.369867,3621.214934
2,BTCUSD-all,0.7,1,16,0.018247,256,0.01,0m 28s,277.049845,966.685962,4131.692223,5161.871618
3,BTCUSD-all,0.7,1,32,0.018247,256,0.01,0m 25s,320.690455,1095.638766,4843.481472,5948.413515
4,BTCUSD-all,0.7,7,4,0.022984,160,0.01,0m 41s,181.824693,485.313485,1108.642757,1606.302584
5,BTCUSD-all,0.7,7,8,0.022984,160,0.01,0m 40s,177.079242,490.131885,987.433506,1473.167303
6,BTCUSD-all,0.7,7,16,0.022984,160,0.01,0m 28s,214.062812,563.116748,1409.866728,1867.897735
7,BTCUSD-all,0.7,7,32,0.022984,160,0.01,0m 32s,191.433417,522.843818,1278.900894,1732.781172
8,BTCUSD-all,0.7,14,4,0.027540,224,0.01,0m 51s,251.721520,599.900674,1892.322415,2397.576325
9,BTCUSD-all,0.7,14,8,0.027540,224,0.01,0m 43s,205.495391,528.872780,1334.416628,1772.189672
